In [467]:
import networkx as nx
import numpy as np
from scipy.linalg import eigvals, eigh
from scipy.sparse.linalg import eigsh
from sklearn.decomposition import PCA
from collections import deque
from sklearn.cluster import KMeans

In [468]:
def prelim_tr(G):
    leaves = [node for node in G.nodes if G.degree(node)==1]
    if len(leaves)>2:
        print("too many leaves,", leaves)
        return None
    H= G.copy()
    queue = deque([node for node in H.nodes() if H.degree(node)==2])
    in_queue = set(queue)
    
    while queue:
        node = queue.popleft()
        in_queue.discard(node)
        
        if node not in H or H.degree(node)!=2:
            continue
            
        u,w = list(H.neighbors(node))
        H.remove_node(node)
        
        if u!=w and not H.has_edge(u,w):
            H.add_edge(u,w)
            
        for n in (u,w):
            if H.has_node(n) and H.degree(n)==2 and n not in in_queue:
                queue.append(n)
                in_queue.add(n)
    return H

In [448]:
def adj_et_deg(G):
    A = nx.to_numpy_array(G)
    degrees = np.sum(A, axis=1)
    return A, degrees

def laplacian_et_signless(A, D):
    L = D-A
    Q = D+A
    return L, Q

In [449]:
def signless_laplacian_rad(Q):
    q_eigs = np.sort(np.real(eigvals(Q)))[::-1]
    q_spectral_radius = q_eigs[0]
    return q_spectral_radius

In [450]:
def fiedler_vect(G):
    L = nx.laplacian_matrix(G).astype(float)
    eigvals, eigvects = eigsh(L, k=2, which='SM')
    
    idx = np.argsort(eigvals)
    
    fiedler_vect = eigvects[:,idx[1]]
    return fiedler_vect

In [451]:
def fiedler_sort(G):
    fvect = fiedler_vect(G)
    nodes = list(G.nodes)
    fdict = {node: val for node, val in zip(nodes, fvect)}
    sorted_tup = sorted(fdict.items(), key=lambda x: x[1])
    return sorted_tup

In [452]:
def gap_threshold(sorted_fiedler, factor):
    gaps = np.diff([t[1] for t in sorted_fiedler])
    median_gap = np.median(gaps)
    std_gap = np.std(gaps)
    exclude_outliers = gaps[gaps <= median_gap+factor*std_gap]
    
    if len(exclude_outliers)==0:
        return gaps, np.mean(gaps)
    threshold = np.mean(exclude_outliers)
    
    #check prints
    print("median gap:", median_gap)
    print("mean gap:", threshold)
    print("number of regular gaps", len(exclude_outliers))
    print("number of irregular gaps", len(gaps)-len(exclude_outliers))
    
    return gaps, threshold

In [453]:
def eigengap_k(eigenvals):# maybe add max nbr candidates in case 98239824u28u98u eigenvals
    gaps = np.diff(eigenvals)
    k = np.argmax(gaps)+1
    other_gaps = np.delete(gaps, k)
    
    if len(other_gaps)>0:
        median_gap = np.median(other_gaps)
        mean_gap = np.mean(other_gaps)
        std_gap = np.std(other_gaps)
    else:
        median_gap=0
        mean_gap = 0
        std_gap = 1
    ratio = k/median_gap if median_gap>0 else np.inf
    z_score = (k- mean_gap)/std_gap
    
    return k, ratio, z_score

In [ ]:
def Q_eigengap(eigenvals):
    gaps = np.diff(eigenvals)
    

In [456]:
# tolerance on multiplicity

In [475]:
def spectral_embedding(G, n_ev):
    A, degrees = adj_et_deg(G)
    D = np.diag(degrees)
    L, Q = laplacian_et_signless(A,D)
    eigenvals_L, eigenvecs_L = eigh(L)
    embedding = eigenvecs_L[:,1:n_ev+1]
    return embedding, eigenvals_L[1:n_ev+1]

In [476]:
def signless_spectral_embedding(G, n_e):
    A, degrees = adj_et_deg(G)
    D = np.diag(degrees)
    L, Q = laplacian_et_signless(A,D)
    eigenvals_Q, eigenvecs_Q = eigsh(Q, k=n_e, which='LA')
    idx = np.argsort(eigenvals_Q)[::-1]
    eigenvals_Q = eigenvals_Q[idx]
    eigenvecs_Q = eigenvecs_Q[:, idx]
    return eigenvals, eigenvecs

In [ ]:
def weighted_signless_fingerprint(G, n_e):
    eigenvals, eigenvecs = signless_spectral_embedding(G,n_e)
    fingerprint = {}
    total_weight = np.sum(eigenvals)
    for i in range(n_components):
        

In [458]:
def pca_reduce(embedding, variance_threshold=0.9):
    pca = PCA()
    full_transformed = pca.fit_transform(embedding)
    cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.searchsorted(cumulative_variance, variance_threshold) + 1
    reduced_embedding = full_transformed[:, :n_components]
    return reduced_embedding, n_components, cumulative_variance

In [474]:
def choose_embedding_and_cluster_method(G, n_ev, clear_gap_threshold=2.0, variance_threshold=0.9):
    embedding, eigenvals = spectral_embedding(G, n_ev)
    k, ratio, z_score = eigengap_k(eigenvals)
    print(f"eigengap: k = {k}, ratio = {ratio}, z_score = {z_score}")
    
    if ratio < clear_gap_threshold and z_score < clear_gap_threshold:
        print("eigengap not clear enough; applying PCA.")
        final_embedding, chosen_dim, cum_var = pca_reduce(embedding, variance_threshold)
        method = 'pca'
    else:
        final_embedding = embedding[:, :k]
        chosen_dim = k
        method = 'eigengap'
    
    print(f"Final embedding dimension: {chosen_dim}")
    return final_embedding, chosen_dim, k, ratio, z_score, method


In [460]:
def signless_embedding(G, cluster_embed):
    A, degrees = adj_et_deg(G)
    D_sqrt_inv = np.diag(1/np.sqrt(np.where(degrees==0, 1e-10, degrees)))
    Q_norm = D_sqrt_inv @ Q @ D_sqrt_inv
    Qr, eigenvals_Qn = signless_laplacian_rad(Q_norm)
    
    augmented_embedding = np.hstack([cluster_embed, Qr])
    return augmented_embedding

In [461]:
def partition_by_gap(G):
    sortedtup = fiedler_sort(G)
    factor = 1
    gaps, threshold = gap_threshold(sortedtup, factor)
    nodes = list(G.nodes())
    
    balls = []
    current_ball = {'vertices':[sortedtup[:][0]],
                    'fiedler values': [sortedtup[:][1]],
    }
    
    for i in range(1, len(sortedtup)):
        if sortedtup[i][1]-sortedtup[i-1][1] > threshold:
            current_ball['size'] = len(current_ball['vertices'])
            current_ball['density'] = 1 #if low cost then just run spectral conds
            balls.append(current_ball)
            current_ball = {'vertices': [], 'fiedler_vals':[]}
        current_ball['vertices'].append(sortedtup[i][0])
        current_ball['fiedler_vals'].append(sortedtup[i][1])
    if current_ball:
        current_ball['size'] = len(current_ball['vertices'])
        current_ball['density'] = 1
        balls.append(current_ball)
    return balls

In [462]:
def ball_edge(ball1, ball2, G):
    edges = []
    for u in ball1['vertices']:
        for v in ball2['vertices']:
            if G.has_edge(u,v):
                edges.append((u,v))
                return True
    return False

In [463]:
def build_ball_graph(balls, G):
    BG = nx.Graph()
    for i, ball in enumerate(balls):
        BG.add_node(i, ball=ball, importance=['density'])
    for i in range(n):
        for j in range(i+1,n):
            if ball_edge(balls[i], balls[j], G):
                BG.add_edge(i,j)
    return BG

In [472]:
import random
from graph_loader import load_all_graphs

folder = 'test_data'
graphs = load_all_graphs(folder)
G = prelim_tr(random.choice(list(graphs.values())))


In [473]:
embedding = spectral_embedding(G, 10)

final_embedding, chosen_dim, k_val, gap_ratio, gap_zscore, method = choose_embedding_and_cluster_method(G, n_ev=10, clear_gap_threshold=2.0, variance_threshold=0.9)
    
print("Chosen method:", method)
print("Final embedding shape:", final_embedding.shape)

original embedding shape: (1998, 10)
original embedding shape: (1998, 10)
eigengap: k = 1, ratio = 314.4290763861398, z_score = 263.15032461003824
Final embedding dimension: 1
Chosen method: eigengap
Final embedding shape: (1998, 1)


/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([0, 0, 0, ..., 0, 0, 0])

In [442]:
clustering(final_embedding, k_val, method)

/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


array([0, 0, 0, ..., 0, 0, 0])

In [413]:
partition_by_gap(G)

median gap: 1.4736257084486297e-05
mean gap: 2.6588473033270144e-05
number of regular gaps 2920
number of irregular gaps 79


[{'vertices': [(2661, -0.0976397188390776)],
  'fiedler values': [(1749, -0.09576830507161635)],
  'size': 1,
  'density': 1},
 {'vertices': [1749],
  'fiedler_vals': [-0.09576830507161635],
  'size': 1,
  'density': 1},
 {'vertices': [2677],
  'fiedler_vals': [-0.09242385349848124],
  'size': 1,
  'density': 1},
 {'vertices': [2675],
  'fiedler_vals': [-0.0910088933252773],
  'size': 1,
  'density': 1},
 {'vertices': [2660],
  'fiedler_vals': [-0.09090003447637143],
  'size': 1,
  'density': 1},
 {'vertices': [1756],
  'fiedler_vals': [-0.08910639141707609],
  'size': 1,
  'density': 1},
 {'vertices': [2678],
  'fiedler_vals': [-0.08735252881053773],
  'size': 1,
  'density': 1},
 {'vertices': [2674],
  'fiedler_vals': [-0.08428891196438829],
  'size': 1,
  'density': 1},
 {'vertices': [2676],
  'fiedler_vals': [-0.08353106875133497],
  'size': 1,
  'density': 1},
 {'vertices': [1754],
  'fiedler_vals': [-0.0791135303033747],
  'size': 1,
  'density': 1},
 {'vertices': [2523],
  'fied